# Using pyAscore for scoring PTM localization

In this document, we will look at the most common command line workflows for scoring phosphosite localizations. We will focus on the specification and meaning of the most important inputs and arguments before discussing the interpretation of results. For an up-to-date explanation of installation instructions, all command line parameters, and how to use the Python API, readers can refer to pyAscore's full documentation at [https://pyascore.readthedocs.io](https://pyascore.readthedocs.io).

## Input to pyAscore

Scoring the localization of post-translational modifications occurs after database search has been performed on a set of spectra. pyAscore takes as input both the spectra file and a file of identifications.

In many cases, users may have a set of spectra files whose searches have been aggregated into a final set of identifications. Before using pyAscore, these identification files should be split up so that each file of IDs only corresponds to a single spectra file. See section 2.3 bellow for an input file format that works well for this situation.

At the folowing github link, users can download a reduced version of one the samples analyzed in the main manuscript, **PXD007740.ctr_2h_R1_IMAC_1.mzML**, which is a representative human phosphoproteome run. This is the file where pyAscore will look for spectra to score. We have already analyzed this file using Comet for database search, and so users should also download the result file from the same link, **PXD007740.ctr_2h_R1_IMAC_1.pep.xml**.

**Tutorial File Link:**

[https://github.com/AnthonyOfSeattle/pyAscoreValidation/tree/main/notebook/pyascore-tutorial](https://github.com/AnthonyOfSeattle/pyAscoreValidation/tree/main/notebook/pyascore-tutorial)

## Command line usage

### Basic command line usage

Once installed, pyAscore can be used with a simple command line call that takes a spectra file, identification file, and output file as positional arguments. By default, pyAscore will accept spectra in the mzML format and PSMs in pepXML format. The modification of interest can be specified by passing the modifiable amino acids with their single letter codes through the `--residues` command line argument and the mass of the modification through the `--mod-mass` command line argument. For example, if a user wants to analyze phosphorylation, they would specify that S (Serine), T (Threonine), and Y (Tyrosine) are modifiable and that 79.9663 Da is the modification mass. This is also the default modification.

**Example 1:**
```
pyascore --residues STY \
         --mod_mass 79.9663 \
         PXD007740.ctr_2h_R1_IMAC_1.mzML \
         PXD007740.ctr_2h_R1_IMAC_1.pep.xml \
         PXD007740.ctr_2h_R1_IMAC_1.pyascore.tsv
```

**Result:**
```
$ ls

PXD007740.ctr_2h_R1_IMAC_1.mzML     PXD007740.ctr_2h_R1_IMAC_1.pyascore.tsv
PXD007740.ctr_2h_R1_IMAC_1.pep.xml
```

### Customizing analysis with command line parameters

**Run pyAscore with different input file types.** In many cases, individuals will have spectra or identifications in other standard file formats. For spectra, users can also supply mzXML files, and for PSMs, users can supply mzIdentML, percolatorTXT, and mokapotTXT. The format of the input files must be specified with the `--spec_file_type` and `--ident_file_type` arguments respectively.

**Example 2:**
```
pyascore --residues STY \
         --mod_mass 79.9663 \
         --spec_file_type mzXML \
         --ident_file_type mzIdentML \
         PXD007740.ctr_2h_R1_IMAC_1.mzXML \
         PXD007740.ctr_2h_R1_IMAC_1.mzid \
         PXD007740.ctr_2h_R1_IMAC_1.pyascore.tsv
```

**Run pyAscore with lower MS2 tolerance:** pyAscore works by looking for fragment ions within the supplied spectra which match a theoretical fragment pattern. Users should tailor the tolerance of this search so that it matches the instrument resolution. This can be done with the `–mz_error option`. For high resolution data, we have been using a tolerance of 0.02 Da, and for low resolution data, we have been using a tolerance of 0.5 Da. These correspond well to the recommended parameters for the Comet database search software, which can be seen at [http://comet-ms.sourceforge.net/](http://comet-ms.sourceforge.net/).

**Example 3:**
```
$ pyascore --residues STY \
           --mod_mass 79.9663 \
           --mz_error 0.5 \
           PXD007740.ctr_2h_R1_IMAC_1.mzML \
           PXD007740.ctr_2h_R1_IMAC_1.pep.xml \
           PXD007740.ctr_2h_R1_IMAC_1.pyascore.tsv
```

**Run pyAscore with different fragment types:** By default, pyAscore will score b and y ion fragment peaks, which are the most abundant peaks for HCD and CID fragmentation data. If a user wants to analyze ETD fragmentation data, it is recommended to score c and z+H ions. This can be specified with the `--fragment_types` option. The Z character is used to differentiate the z+H ion from the z ion.

**Example 4:**
```
$ pyascore --residues STY \
           --mod_mass 79.9663 \
           --fragment_types cZ \
           PXD007740.ctr_2h_R1_IMAC_1.mzML \
           PXD007740.ctr_2h_R1_IMAC_1.pep.xml \
           PXD007740.ctr_2h_R1_IMAC_1.pyascore.tsv
```

**Run pyAscore with neutral losses:** A user has the option to use neutral loss peaks in their scoring procedure, and these can be different for modified and unmodified residues. A user can supply a comma sepparated list of amino acid groups, uppercase for unmodified residues and lowercase for modified, and a comma sepparated list of neutral loss masses. If, for example, a user wants to use the H3P04 neutral loss ions, a loss of 97.976896, on modified Ser, Thr, and Tyr residues, they could use the following command. It should be noted that the use of neutral loss peaks is still experimental, and that none of the datasets in the paper were analyzed with this option specified.

**Example 5:**
```
$ pyascore --residues STY \
           --mod_mass 79.9663 \
           --neutral_loss_groups sty \
           --neutral_loss_masses 97.976896 \
           PXD007740.ctr_2h_R1_IMAC_1.mzML \
           PXD007740.ctr_2h_R1_IMAC_1.pep.xml \
           PXD007740.ctr_2h_R1_IMAC_1.pyascore.tsv
```

### Feeding ID data into pyAscore when your file type is not supported

Often, a user may have used a search engine which doesn’t output a standard format but they still want to connect it to pyAscore. Or they may want to manipulate data before handing it off to pyAscore and want to work with a simpler format for connecting their pipeline than XML based data. This situation is easily handled by the percolatorTXT input format since the standard Percolator output can be reduced to the minimal tab delimited layout below.

```
   scan  rt     sequence                      charge
0  2082  304.8  S[79.9663]NNSNSNSGGK          2
1  2624  402.8  RARES[79.9663]DNEDAK          3
2  2625  402.9  SS[79.9663]NGNESNGAK          2
3  2655  405.4  n[42.010565]S[79.9663]DAGRK   2
```

Once a user makes a tsv file with their PSMs, the data can be fed to pyAscore as though it was a true percolator output file.

**Example 6:**
```
$ pyascore --residues STY \
           --mod_mass 79.9663 \
           --ident_file_type percolatorTXT \
           PXD007740.ctr_2h_R1_IMAC_1.mzML \
           PXD007740.ctr_2h_R1_IMAC_1.tsv \
           PXD007740.ctr_2h_R1_IMAC_1.pyascore.tsv
```

## Understanding pyAscore's output

pyAscore outputs a single .tsv style file with one entry for every PSM containing the modification of interest. For this tutorial, we ran pyAscore on PXD007740.ctr_2h_R1_IMAC_1.mzML and PXD007740.ctr_2h_R1_IMAC_1.pep.xml using the command in **Example 1**. The output data can analyzed with any program or programming language which reads tab delimited files.

In [1]:
cd pyascore-tutorial

/net/villen/vol2/users/valenta4/pyAscoreValidation/notebook/pyascore-tutorial


In [2]:
import pandas as pd

In [3]:
pyascore_output = pd.read_csv(
    "PXD007740.ctr_2h_R1_IMAC_1.pyascore.tsv",
    sep="\t"
)
print(
    "Columns and datatypes:\n{}".format(
        pyascore_output.dtypes)
)

Columns and datatypes:
Scan                   int64
LocalizedSequence     object
PepScore             float64
Ascores               object
AltSites              object
dtype: object


The output columns of pyAscore can be broken up into two parts. The first part is the final form of the PSMs for a given dataset, i.e. the peptide sequences with re-arranged modifications and the scans which they correspond to.

- **Scan:** This is the scan number from the supplied spectra file. It is usually taken from the scan header and so care should by taken that this matches expectations.

- **LocalizedSequence:** This is the modified peptide with the PTM of interest placed in the best positions according the the PepScore. All outputed masses are rounded to their whole number representations.

In [4]:
pyascore_output.iloc[1000:1020, :2]

,Scan,LocalizedSequence
1000,6771,NFILDQC[57]NVY[80]NSGQRRK
1001,6773,RKHQGC[57]SVS[80]FQLEK
1002,6774,AS[80]S[80]SPSPT[80]QPVLGAQPHSR
1003,6776,EEKAT[80]M[16]KNVPS[80]R
1004,6777,SREHPHS[80]RS[80]PS[80]PEPR
1005,6779,TESLENQLVS[80]VMNEMQK
1006,6781,SGLVQC[57]GET[80]AWHGGK
1007,6782,HALAADRLLQNAEEM[16]TGDET[80]AIGNIS[80]LVK
1008,6784,ILIPFIPAFYINQSELVLS[80]HK
1009,6786,RRS[80]PPPGPDGHAK


The second set of columns corresponds to the actual scores which come along with the localized sequences. These represent how confident a user should be in the peptide sequence's final form.

- **PepScore:** This score gives the total amount of evidence for the listed sequence being correct. It is based on the total number of matching theoretical ions to the ranked peaks within the supplied spectrum. A value of inf means that there is no ambiguity in the localized sequence.

- **Ascores:** This semicolon separated list of scores gives the relative amount of evidence for the localization of the modification of interest vs the next best localization. It is based on the number of matching theoretical site determining ion peaks in the supplied spectrum. A value of inf means that there is no ambiguity in the site placement. There is one entry in the list per modification of interest on the peptide.

- **Altsites:** This semicolon separated list of comma separated positions gives the next best locations for a modification. There is one list of alternative sites per modification of interest on the peptide.


In [5]:
pyascore_output.iloc[1000:1020, 2:]

,PepScore,Ascores,AltSites
1000,41.231426,61.68848,12
1001,3.477892,0.0,7
1002,1.555447,0.0;0.0;1.3276455,"4,6;4,6;4,6"
1003,inf,inf;inf,;
1004,7.417679,5.2201395;2.8990664;3.8300576,1;1;1
1005,4.110435,3.235395,1
1006,0.703247,1.3716472,1
1007,0.487868,5.5877275;2.5625544,16;16
1008,0.655502,5.1771717,"10,14"
1009,inf,inf,NaN


Most users will be interested in the Ascores for a given site. These are defined on a scale which corresponds to a negative log p-value, which means that increasing scores are better. Several cuttoffs may be of interest to the user:

- 20 (Theoretical confidence of 99%)

- 13 (Theoretical confidence of 95%)

- 10 (Theoretical confidence of 90%)

Within our lab, we tend to use a cutoff of 13, although the final choice will require considering the balance between false localization rate and number of localized PSMs.

**Note:** While pyAscores output can be analyzed on its own. Usually, it is best to filter PSMs to the desired FDR level (e.g. 1%) and then use the localization scores as a secondary filter.